In [1]:
import numpy as np
import matplotlib.pyplot as plt

from termcolor import colored, cprint

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torch
import torchvision
import torchvision.transforms as transforms

from torch.utils.data.dataset import Dataset
from torchvision import transforms

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

print(device)

cpu


In [24]:
 print(torch.__version__)

1.0.1.post2


In [2]:
Data = np.loadtxt('dataWhole.txt', dtype=str, delimiter = '\n')


In [15]:
print(Data.shape)
separate = int(Data.shape[0] * 0.8)

(1941123,)


In [16]:
dataTrain = Data[0:separate]
dataValid = Data[separate:]

In [3]:
blackTurnBoard = np.ones(shape=(15, 15), dtype=np.int8)
whiteTurnBoard = -np.ones(shape=(15, 15), dtype=np.int8)

In [4]:
def toTurn(turn):
    letter = ord(turn[0]) - ord('a')
    num = int(turn[1:]) - 1
    return letter, num

In [5]:
def toBoards(game, color, turn):
    if color == "black":
        playerBoard = blackTurnBoard
    else:
        playerBoard = whiteTurnBoard
        
    curPlayer = 1
    whiteBoard = np.zeros(shape=(15, 15), dtype=np.int8)
    blackBoard = np.zeros(shape=(15, 15), dtype=np.int8)
    
    for i in range(1, turn + 1):
        turnC = toTurn(game[i])
        if curPlayer == 1:
            blackBoard[turnC] = 1
        else:
            whiteBoard[turnC] = -1
        curPlayer *= -1
    curBoard = np.zeros(shape=(3, 15, 15))
    curBoard[0,:] = playerBoard
    curBoard[1,:] = blackBoard
    curBoard[2,:] = whiteBoard
    label = toTurn(game[turn + 1])
    return curBoard, label

In [6]:
def printBoard(board):
    for i in range(14, -1, -1):
        for j in range(15):
            if board[1, i, j] != 0:
                print(" 1",  end='')
            elif board[2, i, j] != 0:
                print("-1", end='')
            else:
                print(" 0", end='')
        print()
    print()

In [33]:
def takeRandom(data):
    game = []
    while len(game) <= 4:
        gameNum = np.random.randint(data.shape[0])
        game = data[gameNum].split()
    color = game[0]
    turn = np.random.randint((len(game) - 2) // 2)
    if color[0] == 'b':
        board, label = toBoards(game, color, turn * 2)
    else:
        print(1)
        board, label = toBoards(game, color, turn * 2 + 1)

    return board, label[0] * 15 + label[1]

In [34]:
def makeBatch(size, data):
    batch = np.zeros(shape=(size, 3, 15, 15))
    labels = np.zeros(shape=size, dtype=np.long)
    for i in range(size):
        batch[i,:,:,:], labels[i] = takeRandom(data)
    return torch.from_numpy(batch), torch.from_numpy(labels)

In [35]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.batchnorm = nn.BatchNorm2d(3, affine=False)
        self.pad2 = nn.ConstantPad2d(2, 0)
        self.pad1 = nn.ConstantPad2d(1, 0)

        self.conv1 = nn.Conv2d(3, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(64, 32, 3)
        self.conv4 = nn.Conv2d(32, 16, 3)
        self.conv5 = nn.Conv2d(16, 1, 5)

    def forward(self, x):
        x = self.batchnorm(x.float())
        x = self.pad1(F.relu(self.conv1(x)))
        x = self.pad1(F.relu(self.conv2(x)))
        x = self.pad1(F.relu(self.conv3(x)))
        x = self.pad1(F.relu(self.conv4(x)))
        x = self.pad2(F.relu(self.conv5(x)))

        x = x.view(-1, 225)
 
        return x

In [36]:
net = Net()
net = net.to(device)
criterion = nn.CrossEntropyLoss()
batchNum = 1000000
epochNum = 5
batchSize = 32
trainPart = 0.8
validPart = 0.2
optimizer = optim.Adam(net.parameters(), lr=0.0001)
#optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [37]:
for epoch in range(epochNum):  # loop over the dataset multiple times
    running_loss = 0.0
    for i in range(batchNum):
        # get the inputs
        inputs, labels = makeBatch(batchSize, Data)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0
        if i % 200000 == 0:
            torch.save(net.state_dict(), str(i + epoch) + "NN.txt")

print('Finished Training')

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


KeyboardInterrupt: 

In [23]:
torch.save(net.state_dict(), "finalNN.txt")


## Validation

In [17]:
correct = 0
total = 0
for i in range(100000):
    if i % 1000 == 999:
        print('Accuracy of the network on the 1000 test images: %5f %%' % (100 * correct / total))    
    inputs, labels = makeBatch(batchSize, dataTrain)
    outputs = net(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 100000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 1000 test images: 41.898148 %
Accuracy of the network on the 1000 test images: 41.963169 %
Accuracy of the network on the 1000 test images: 42.033803 %
Accuracy of the network on the 1000 test images: 41.999562 %


KeyboardInterrupt: 

## Game

In [18]:
def toTripleBoard(board, side):
    curBoard = np.zeros(shape=(3, 15, 15), dtype=np.float)
    blackBoard = np.zeros(shape=(15, 15), dtype=np.float)
    whiteBoard = np.zeros(shape=(15, 15), dtype=np.float)

    if side == 1:
        blackTurnBoard = np.ones(shape=(15, 15), dtype=np.float)
    else:
        blackTurnBoard = -np.ones(shape=(15, 15), dtype=np.float)
    curBoard[0,:] = blackTurnBoard
    for i in range(15):
        for j in range(15):
            if board[i, j] == 1:
                blackBoard[i, j] = 1
            if board[i, j] == -1:
                whiteBoard[i, j] = -1
    curBoard[1,:] = blackBoard
    curBoard[2,:] = whiteBoard
    return curBoard


In [19]:
def printBoard(curBoard):
    print("----------------------------------------------------------------")
    cprint("     0   1   2   3   4   5   6   7   8   9   10  11  12  13  14 ", "blue")
    for i in range(15):
        if i < 10:
            cprint(" " + str(i), "blue", end=" ")
        else:
            cprint(i, "blue", end=" ")
        for j in range(15):
            if curBoard[i, j] == -1:
                print("|", end="")
                cprint(str(-1), 'red', "on_green", attrs=['blink'], end=" ")
            else:
                print("|", end = " ")
                if curBoard[i, j] == 1:
                    cprint(int(curBoard[i, j]), 'yellow', "on_blue", attrs=['blink'], end=" ")
                else:
                    print(int(curBoard[i, j]), end=" ")
        print("|")
        print("----------------------------------------------------------------")
    print()
    return


In [25]:
def isGg(board, color):
    if color == 0:
        size = 4
    else:
        size = -4
    for i in range(15):
        for j in range(10):
            if board[i, j] + board[i, j + 1] + board[i, j + 2] + board[i, j + 3] + board[i, j + 4] == size:
                for k in range(5):
                    if board[i, j + k] == 0:
                        return i, j + k

    for i in range(10):
        for j in range(15):
            if board[i, j] + board[i + 1, j] + board[i + 2, j] + board[i + 3, j] + board[i + 4, j] == size:
                for k in range(5):
                    if board[i + k, j] == 0:
                        return i + k, j
    
    for i in range(10):
        for j in range(10):
            if board[i, j] + board[i + 1, j + 1] + board[i + 2, j + 2] + board[i + 3, j + 3] + board[i + 4, j + 4] == size:
                for k in range(5):
                    if board[i + k, j + k] == 0:
                        return i + k, j + k
    for i in range(5, 15):
        for j in range(10):
            if board[i, j] + board[i - 1, j + 1] + board[i - 2, j + 2] + board[i - 3, j + 3] + board[i - 4, j + 4] == size:
                for k in range(5):
                    if board[i - k, j + k] == 0:
                        return i - k, j + k
    return (-1, -1)

 ## (we are white)

In [26]:
curBoard = np.zeros(shape=(15, 15), dtype=np.float)

while True:
    board = toTripleBoard(curBoard, 1)
    with torch.no_grad():
        outputs = net(torch.unsqueeze(torch.from_numpy(board), 0))
        _, netTurn = torch.max(outputs, 1)
        turnX, turnY = netTurn // 15, netTurn % 15

        while curBoard[turnX, turnY] != 0:
            outputs[netTurn] = 0
            _, netTurn = torch.max(outputs, 1)
            turnX, turnY = netTurn // 15, netTurn % 15
    ggTest = isGg(curBoard, 0)
    if ggTest != (-1, -1):
        turnX, turnY = ggTest
        print(1)
    
    curBoard[turnX, turnY] = 1
    printBoard(curBoard)
    
    x, y = list(map(int, input().split()))
    curBoard[x, y] = -1
    #printBoard(curBoard)


----------------------------------------------------------------
     0   1   2   3   4   5   6   7   8   9   10  11  12  13  14 
 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 3 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 4 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 5 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 6 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------

KeyboardInterrupt: 

## (we are black)

In [21]:
curBoard = np.zeros(shape=(15, 15), dtype=np.float)

while True:
    x, y = list(map(int, input().split()))
    curBoard[x, y] = 1
    #printBoard(curBoard)
    
    board = toTripleBoard(curBoard, 1)
    with torch.no_grad():
        outputs = net(torch.unsqueeze(torch.from_numpy(board), 0))
        _, netTurn = torch.max(outputs, 1)
        turnX, turnY = netTurn // 15, netTurn % 15
        while curBoard[turnX, turnY] != 0:
            outputs[netTurn] = 0
            _, netTurn = torch.max(outputs, 1)
            turnX, turnY = netTurn // 15, netTurn % 15

    ggTest = isGg(curBoard, 1)
    if ggTest != (-1, -1):
        turnX, turnY = ggTest
        print(1)

    curBoard[turnX, turnY] = -1
    printBoard(curBoard)


6 6
----------------------------------------------------------------
     0   1   2   3   4   5   6   7   8   9   10  11  12  13  14 
 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 3 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 4 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 5 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 6 | 0 | 0 | 0 | 0 | 0 |-1 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
---------------------

KeyboardInterrupt: 

### formated.

In [27]:
def toTurn(turn):
    letter = ord(turn[0]) - ord('a')
    num = int(turn[1:]) - 1
    return letter, num


In [28]:
gameStr = input().split()
curBoard = np.zeros(shape=(15, 15), dtype=np.float)

g6


In [29]:
for i in range(len(gameStr)):
    x, y = toTurn(gameStr[i])
    if i % 2 == 0:
        curBoard[x, y] = 1
    else:
        curBoard[x, y] = -1


In [30]:
color = len(gameStr) % 2

In [31]:
board = toTripleBoard(curBoard, 1)
with torch.no_grad():
    outputs = net(torch.unsqueeze(torch.from_numpy(board), 0))
    _, netTurn = torch.max(outputs, 1)
    netTurn = int(netTurn)
    turnX, turnY = netTurn // 15, netTurn % 15
    while curBoard[turnX, turnY] != 0:
        outputs[netTurn] = 0
        _, netTurn = torch.max(outputs, 1)
        turnX, turnY = netTurn // 15, netTurn % 15

ggTest = isGg(curBoard, color)
if ggTest != (-1, -1):
    turnX, turnY = ggTest


In [32]:
print(chr(ord('a') + netTurn // 15), netTurn % 15, sep="")

g8
